<a href="https://colab.research.google.com/github/paul20010801/my-python/blob/master/%E5%AF%A6%E7%BF%92TVBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json

In [12]:
import requests
from bs4 import BeautifulSoup
import time
import json

import requests
from bs4 import BeautifulSoup
import time
import json

class TVBS:

    def __init__(self, page: int, start_url: str, start_id: int) -> None:
        self.page = page  # 爬的文章數
        self.start_url = start_url  # 開始的URL
        self.start_id = start_id  # 開始的文章ID
        self.article_list = []

    def fetch_data(self, url: str):
        try:
            response = requests.get(url)
            response.encoding = 'utf-8'  # 設置為 utf-8
            soup = BeautifulSoup(response.text, 'html.parser')
            script_tag = soup.find('script', type="application/ld+json")
            main_data = json.loads(script_tag.string) if script_tag else None

            return main_data, soup  # 返回解析好的 JSON 和 BeautifulSoup 物件

        except requests.RequestException as e:
            print(f"Error fetching data from {url}: {e}")
            return None, None

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {url}: {e}")
            return None, None

    def get_category(self, main_data: dict):
        """從抓取的數據中提取文章的分類"""
        return main_data.get("articleSection", "No category available") if main_data else None

    def get_title(self, main_data: dict):
        """從抓取的數據中提取文章的標題"""
        return main_data.get("headline", "No title available") if main_data else None

    def get_datetime(self, main_data: str):
        """從抓取的數據中提取文章的發佈日期（僅日期部分）"""
        full_datetime = main_data.get("datePublished", "No date available") if main_data else "No date available"
        if full_datetime != "No date available":
            return full_datetime[:10]  # 只取 "YYYY-MM-DD" 部分
        else:
            return "No date available"

    def get_link(self, url: str):
        """返回文章的URL"""
        return url

    def get_content(self, main_data: dict):
        """從抓取的數據中提取文章的正文內容"""
        return main_data.get("articleBody", "No content available") if main_data else None

    def get_comments(self, soup: BeautifulSoup):
        """從抓取的數據中提取文章的留言"""
        comments_section = soup.find('div', class_='comments')  # 假設留言位於某個特定的 div 中
        if comments_section:
            comments = [comment.text.strip() for comment in comments_section.find_all('p')]
            return comments if comments else "No comments available"
        return "No comments available"

    def get_info(self):
        """循環抓取文章信息並輸出"""
        for i in range(self.start_id - self.page, self.start_id):
            article_url = self.start_url + str(i)
            main_data, soup = self.fetch_data(article_url)

            if main_data:
                # 提取所需信息
                category = self.get_category(main_data)
                title = self.get_title(main_data)
                datetime = self.get_datetime(main_data)
                link = self.get_link(article_url)
                content = self.get_content(main_data)
                comments = self.get_comments(soup)

                # 將提取到的數據存儲到 article_list 中
                article_info = {
                    "category": category,
                    "title": title,
                    "datetime": datetime,
                    "link": link,
                    "content": content[:100] + "...",  # 只顯示前100個字符作為摘要
                    "comments": comments
                }

                self.article_list.append(article_info)

                # 打印輸出每篇文章的提取數據
                print(f"Article ID {i}:")
                print(f"Category: {category}")
                print(f"Title: {title}")
                print(f"Datetime: {datetime}")
                print(f"Link: {link}")
                print(f"Content: {content[:100]}...")  # 顯示內容的前100個字符作為摘要
                print(f"Comments: {comments}\n")

            time.sleep(0.3)  # 爬取間隔
        return None

# 創建 TVBS 對象並抓取文章信息
tvbs = TVBS(50, "https://news.tvbs.com.tw/money/", 2628359)
tvbs.get_info()


Article ID 2628310:
Category: 理財房地產
Title: 網戀布萊德彼特是騙局？西班牙2女遭冒牌小布詐騙千萬│TVBS新聞網
Datetime: 2024-09-24
Link: https://news.tvbs.com.tw/money/2628310
Content: 西班牙警方今天表示，他們逮捕了5名嫌疑人，這些人被控透過網路和通訊軟體WhatsApp冒充好萊塢明星布萊德彼特，向兩名女性詐騙32萬5000歐元（約新台幣1173萬元）。法新社報導，西班牙瓜迪亞（Gu...
Comments: No comments available

Article ID 2628311:
Category: 理財房地產
Title: 屏東內埔鐵皮屋起火！「黑煙狂竄」壟罩　1人燒燙傷急送醫　│TVBS新聞網
Datetime: 2024-09-24
Link: https://news.tvbs.com.tw/money/2628311
Content: 今（24日）上午7時19分，位於屏東縣內埔鄉和興路與和美路口的一處鐵皮屋發生火警。屏東縣政府消防局獲報後，立刻出動12車、26人前往現場搶救，一名50歲女子被順利救出，手臂有兩處燒燙傷，所幸意識清楚隨...
Comments: No comments available

Article ID 2628312:
Category: 理財房地產
Title: 蘋果AI功能將分階段釋出！外媒曝光時間表　明年3月可完整體驗│TVBS新聞網
Datetime: 2024-09-24
Link: https://news.tvbs.com.tw/money/2628312
Content: 隨著蘋果發表iPhone16以及發布iOS18之後，全球果迷都在期待官方主打的AppleIntelligence功能何時才會上線？目前已知首波功能會在10月跟著iOS18.1一起推出，而最近外媒《彭博...
Comments: No comments available

Article ID 2628313:
Category: 理財房地產
Title: iPhone 4S被鎖10年！陸妹子解開了　千萬人隔空見證│TVBS新聞網
Datetime: 2024-09-24
Link: https://news.tvbs